This notebook is used to evaluate the cleaned outputs (predicates) provided by Louise once the positive and negative examples, the background knowledge and the metarules for each concept were provided to the system. 

# Settings

In [2]:
!sudo apt install swi-prolog
!pip install pyswip

Password:
sudo: a password is required
  Using cached pyswip-0.2.10-py2.py3-none-any.whl (27 kB)


In [69]:
from pyswip import Prolog
prolog = Prolog()

# Data

In [70]:
with open("clean_output_r.txt", "r") as file:
	predicates = file.readlines()
with open("test.txt", "r") as file:
	test = file.readlines()
with open("test_clear.txt", "r") as file:
	test_clear = file.readlines()

In [71]:
for i in range(0, len(predicates)):
  predicates[i] = predicates[i].split(";")
  predicates[i][len(predicates[i])-1] = predicates[i][len(predicates[i])-1].replace("\n","")
for i in range(0, len(test)):
  test[i] = test[i].split(";")
  test[i][len(test[i])-1] = test[i][len(test[i])-1].replace("\n","")
for i in range(0, len(test_clear)):
  test_clear[i] = test_clear[i].split(";")
  test_clear[i][len(test_clear[i])-1] = test_clear[i][len(test_clear[i])-1].replace("\n","")

In [72]:
 #Function to build the pl file to consult with the BK functions + the predicates returned by louise
 def build_pl(file, predicates):
  data = "" 
  # Reading data from first file 
  with open(file) as fp: 
      data = fp.read() 
  # Merging two files into another file 
  data += "\n\n"
  for i in range(0, len(predicates)):
    data += predicates[i]
    data += "\n"
  with open ('aux_file.pl', 'w') as fp: 
    fp.write(data)
    fp.close()

In [73]:
#Return solutions based on the predicates returned by louise
numConcepts = 8
numBatches = 3
numTests = 5

results = [[]]*numConcepts
results_clear = [[]]*numConcepts

for c in range(0, numConcepts):
  results[c] = [[]]*numBatches
  results_clear[c] = [[]]*numBatches
  for b in range(0, numBatches):
    results[c][b] = [[]]*len(test[c])
    results_clear[c][b] = [[]]*len(test_clear[c])
    pred_aux = predicates[(c*numBatches)+b]
    if pred_aux == ['']:
      for t in range(0,len(test[c])):
        results[c][b][t] = []
      for t in range(0,len(test_clear[c])):
        results_clear[c][b][t] = []
    else:
      build_pl("eval.pl",predicates[(c*numBatches)+b])
      prolog.consult("aux_file.pl")
      for t in range(0,len(test[c])):
        results[c][t][b] = list(prolog.query(test[c][t]))
      for t in range(0,len(test_clear[c])):
        results_clear[c][b][t] = list(prolog.query(test_clear[c][t]))

In [74]:
#Postprocess results
for c in range(0, numConcepts):
  for b in range(0, numBatches):
    for t in range(0,len(test[c])):
      for l in range(0, len(results[c][b][t])):
        if isinstance(results[c][b][t][l], dict):
          list_aux = [str(thing) for thing in results[c][b][t][l]['B']]
          results[c][b][t][l] = "".join(list_aux)
    for t in range(0,len(test_clear[c])):
      for l in range(0, len(results_clear[c][b][t])):
        if isinstance(results_clear[c][b][t][l], dict):
          list_aux = [str(thing) for thing in results_clear[c][b][t][l]['B']]
          results_clear[c][b][t][l] = "".join(list_aux)  


In [ ]:
import copy

correctAnswers = [['0','1','0','1','0'],['1','1','0','0','-'],['1','-','1','-','1'], ['011', '011', '011', '1', '1'], ['0', '1', '-', '1', '-'], ['00', '-', '-', '-', '-'], ['1', '0', '0', '-', '-'], ['-', '-', '1', '1', '-']]

correctAnswers_emp = copy.deepcopy(correctAnswers)

remove_pos = [[],[4],[1,3],[],[2,4],[1,2,3,4],[3,4],[0,1,4]]

for i in range(0, len(remove_pos)):
    remove_pos[i].reverse()
    for j in range(0,len(remove_pos[i])):
        del correctAnswers_emp[i][remove_pos[i][j]]

# Evaluation

## Louise-exp

### All text examples

In [86]:
import statistics as st

accuracies_exp = [[]]*numConcepts
binary_exp = [[]]*numConcepts

#Calculate the accuracies
for c in range(0, numConcepts):
  accuracies_exp[c] = [0]*numBatches
  for b in range(0, numBatches):
    binary_exp[c][b] = [[]]*numTests
    for t in range(0, numTests):
      binary_exp[c][b][t] = [""]*len(results[c][b][t])
      for l in range(0, len(results[c][b][t])):
        if results[c][b][t][l] == correctAnswers[c][t]:
          accuracies_exp[c][b] = accuracies[c][b] + (1/numTests)/len(results[c][b][t])
          binary_exp[c][b][t][l] = 1
        else:
          binary_exp[c][b][t][l] = 0
          

for c in range(0, numConcepts):
  for b in range(0, numBatches):
    print("Louise Accuracy for C" + str(c+1) + " in Phase " + str(b+1) + ": " + str(accuracies_exp[c][b]*100))
    

for c in range(0, numConcepts):
  for b in range(0, numBatches):
    for t in range(0, numTests):
      if binary_exp[c][b][t] == []:
        binary_exp[c][b][t] = 0
      elif not isinstance(binary_exp[c][b][t],int):
        binary_exp[c][b][t] = st.mean(binary_exp[c][b][t])

Louise Accuracy for C1 in Phase 1: 43.52941176470592
Louise Accuracy for C1 in Phase 2: 64.00000000000003
Louise Accuracy for C1 in Phase 3: 96.66666666666667
Louise Accuracy for C2 in Phase 1: 61.11111111111109
Louise Accuracy for C2 in Phase 2: 78.57142857142857
Louise Accuracy for C2 in Phase 3: 95.00000000000003
Louise Accuracy for C3 in Phase 1: 0
Louise Accuracy for C3 in Phase 2: 20.0
Louise Accuracy for C3 in Phase 3: 20.0
Louise Accuracy for C4 in Phase 1: 0
Louise Accuracy for C4 in Phase 2: 40.0
Louise Accuracy for C4 in Phase 3: 40.0
Louise Accuracy for C5 in Phase 1: 20.0
Louise Accuracy for C5 in Phase 2: 0
Louise Accuracy for C5 in Phase 3: 13.333333333333334
Louise Accuracy for C6 in Phase 1: 0
Louise Accuracy for C6 in Phase 2: 0
Louise Accuracy for C6 in Phase 3: 0
Louise Accuracy for C7 in Phase 1: 60.00000000000001
Louise Accuracy for C7 in Phase 2: 60.00000000000001
Louise Accuracy for C7 in Phase 3: 60.00000000000001
Louise Accuracy for C8 in Phase 1: 30.000000000

### No empty strings

In [75]:
import statistics as st

accuracies_exp_emp = [[]]*numConcepts
binary_exp_emp = [[]]*numConcepts

#Calculate the accuracies
for c in range(0, numConcepts):
  accuracies_exp_emp[c] = [0]*numBatches
  binary_exp_emp[c] = [[]]*numBatches
  for b in range(0, numBatches):
    binary_exp_emp[c][b] = [[]]*len(test_clear[c])
    for t in range(0, len(test_clear[c])):
      binary_exp_emp[c][b][t] = [""]*len(results_clear[c][b][t])
      for l in range(0, len(results_clear[c][b][t])):
        if results_clear[c][b][t][l] == correctAnswers_emp[c][t]:
          accuracies_exp_emp[c][b] = accuracies_exp_emp[c][b] + (1/len(correctAnswers_emp[c]))/len(results_clear[c][b][t])
          binary_exp_emp[c][b][t][l] = 1
        else:
          binary_exp_emp[c][b][t][l] = 0



for c in range(0, numConcepts):
  for b in range(0, numBatches):
    print("Louise Accuracy for C" + str(c+1) + " in Phase " + str(b+1) + ": " + str(accuracies_exp_emp[c][b]*100))
    

for c in range(0, numConcepts):
  for b in range(0, numBatches):
    for t in range(0, numTests):
      if binary_exp_emp[c][b][t] == []:
        binary_exp_emp[c][b][t] = 0
      elif not isinstance(binary_exp_emp[c][b][t],int):
        binary_exp_emp[c][b][t] = st.mean(binary_exp_emp[c][b][t])



Louise Accuracy for C1 in Phase 1: 43.52941176470592
Louise Accuracy for C1 in Phase 2: 64.00000000000003
Louise Accuracy for C1 in Phase 3: 96.66666666666667
Louise Accuracy for C2 in Phase 1: 72.22222222222216
Louise Accuracy for C2 in Phase 2: 87.49999999999997
Louise Accuracy for C2 in Phase 3: 100.0
Louise Accuracy for C3 in Phase 1: 0
Louise Accuracy for C3 in Phase 2: 33.33333333333333
Louise Accuracy for C3 in Phase 3: 33.33333333333333
Louise Accuracy for C4 in Phase 1: 0
Louise Accuracy for C4 in Phase 2: 40.0
Louise Accuracy for C4 in Phase 3: 40.0
Louise Accuracy for C5 in Phase 1: 0
Louise Accuracy for C5 in Phase 2: 0
Louise Accuracy for C5 in Phase 3: 22.22222222222222
Louise Accuracy for C6 in Phase 1: 0
Louise Accuracy for C6 in Phase 2: 0
Louise Accuracy for C6 in Phase 3: 0
Louise Accuracy for C7 in Phase 1: 100.0
Louise Accuracy for C7 in Phase 2: 100.0
Louise Accuracy for C7 in Phase 3: 100.0
Louise Accuracy for C8 in Phase 1: 0
Louise Accuracy for C8 in Phase 2: 0

In [79]:
#Return the most repeated element in a list or a list with the most repeated elements
def most_frequent(passed_list, seed): 
  counter = 0 
  num = ""   
  for i in passed_list: 
    curr_frequency = passed_list.count(i) 
    if curr_frequency > counter: 
      counter = curr_frequency
      num = i 
    elif curr_frequency == counter:
      if isinstance(num, list):
        if not i in num:
          num.append(i)
      elif i != num:
        num = [num,i]

  if isinstance(num, list):
    rn.seed(seed)
    num = rn.choice(num)
  
  return num

## Louise-ens

### All test examples

In [84]:
import statistics as st
import random as rn
import copy

accuracies_ens = [[]]*numConcepts
binary_ens = [[]]*numConcepts

resultsAux = copy.deepcopy(results)

for c in range(0, numConcepts):
  for b in range(0, numBatches):
    for t in range(0, numTests):
      if resultsAux[c][b][t] != []:
        resultsAux[c][b][t] = most_frequent(resultsAux[c][b][t],seed)


#Calculate the accuracies
for c in range(0, numConcepts):
  accuracies_ens[c] = [0]*numBatches
  binary_ens[c] = [[]]*numBatches
  for b in range(0, numBatches):
    binary_ens[c][b] = [""]*numTests
    for t in range(0, numTests):
      if resultsAux[c][b][t] == correctAnswers[c][t]:
        accuracies_ens[c][b] = accuracies_ens[c][b] + (1/numTests)
        binary_ens[c][b][t] = 1
      else:
        binary_ens[c][b][t] = 0

for c in range(0, numConcepts):
  for b in range(0, numBatches):
    print("Louise Accuracy for C" + str(c+1) + " in Batch " + str(b+1) + ": " + str(accuracies_ens[c][b]*100))

Louise Accuracy for C1 in Phase 1: 60.00000000000001
Louise Accuracy for C1 in Phase 2: 80.0
Louise Accuracy for C1 in Phase 3: 100.0
Louise Accuracy for C2 in Phase 1: 40.0
Louise Accuracy for C2 in Phase 2: 60.00000000000001
Louise Accuracy for C2 in Phase 3: 100.0
Louise Accuracy for C3 in Phase 1: 0
Louise Accuracy for C3 in Phase 2: 20.0
Louise Accuracy for C3 in Phase 3: 20.0
Louise Accuracy for C4 in Phase 1: 0
Louise Accuracy for C4 in Phase 2: 40.0
Louise Accuracy for C4 in Phase 3: 40.0
Louise Accuracy for C5 in Phase 1: 20.0
Louise Accuracy for C5 in Phase 2: 0
Louise Accuracy for C5 in Phase 3: 20.0
Louise Accuracy for C6 in Phase 1: 0
Louise Accuracy for C6 in Phase 2: 0
Louise Accuracy for C6 in Phase 3: 0
Louise Accuracy for C7 in Phase 1: 60.00000000000001
Louise Accuracy for C7 in Phase 2: 60.00000000000001
Louise Accuracy for C7 in Phase 3: 60.00000000000001
Louise Accuracy for C8 in Phase 1: 60.00000000000001
Louise Accuracy for C8 in Phase 2: 60.00000000000001
Louis

### No empty strings

In [81]:
import statistics as st
import random as rn
import copy

accuracies_ens_emp = [[]]*numConcepts
binary_ens_emp = [[]]*numConcepts

resultsAux = copy.deepcopy(results_clear)


for c in range(0, numConcepts):
  for b in range(0, numBatches):
    for t in range(0, numTests):
      if resultsAux[c][b][t] != []:
        resultsAux[c][b][t] = most_frequent(resultsAux[c][b][t],seed)


#Calculate the accuracies
for c in range(0, numConcepts):
  accuracies_ens_emp[c] = [0]*numBatches
  binary_ens_emp[c] = [[]]*numBatches
  for b in range(0, numBatches):
    binary_ens_emp[c][b] = [""]*len(test_clear[c])
    for t in range(0, len(test_clear[c])):
      if resultsAux[c][b][t] == correctAnswers_emp[c][t]:
        accuracies_ens_emp[c][b] = accuracies_ens_emp[c][b] + (1/len(test_clear[c]))
        binary_ens_emp[c][b][t] = 1
      else:
        binary_ens_emp[c][b][t] = 0

for c in range(0, numConcepts):
  for b in range(0, numBatches):
    print("Louise Accuracy for C" + str(c+1) + " in Batch " + str(b+1) + ": " + str(accuracies_ens_emp[c][b]*100))

Louise Accuracy for C1 in Phase 1: 60.00000000000001
Louise Accuracy for C1 in Phase 2: 80.0
Louise Accuracy for C1 in Phase 3: 100.0
Louise Accuracy for C2 in Phase 1: 50.0
Louise Accuracy for C2 in Phase 2: 75.0
Louise Accuracy for C2 in Phase 3: 100.0
Louise Accuracy for C3 in Phase 1: 0
Louise Accuracy for C3 in Phase 2: 33.33333333333333
Louise Accuracy for C3 in Phase 3: 33.33333333333333
Louise Accuracy for C4 in Phase 1: 0
Louise Accuracy for C4 in Phase 2: 40.0
Louise Accuracy for C4 in Phase 3: 40.0
Louise Accuracy for C5 in Phase 1: 0
Louise Accuracy for C5 in Phase 2: 0
Louise Accuracy for C5 in Phase 3: 33.33333333333333
Louise Accuracy for C6 in Phase 1: 0
Louise Accuracy for C6 in Phase 2: 0
Louise Accuracy for C6 in Phase 3: 0
Louise Accuracy for C7 in Phase 1: 100.0
Louise Accuracy for C7 in Phase 2: 100.0
Louise Accuracy for C7 in Phase 3: 100.0
Louise Accuracy for C8 in Phase 1: 0
Louise Accuracy for C8 in Phase 2: 0
Louise Accuracy for C8 in Phase 3: 0
